### Import Libraries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import os

### Load in dataset
The data is a mock dataset generated using ChatGPT for demonstration and analysis purposes

In [6]:
data = pd.read_csv('arcteryx_data_2022_2024.csv')
df = pd.DataFrame(data)

### Dataset Overview

In [7]:
# Check Dataset Dimensions
df.shape

(10500, 15)

In [8]:
# View Column Names
df.columns

Index(['OrderDate', 'CustomerID', 'OrderID', 'ProductID', 'ProductName',
       'Category', 'Gender', 'Age', 'Quantity', 'ExpectedPrice', 'FinalPrice',
       'Discount', 'Rating', 'Region', 'PaymentMethod'],
      dtype='object')

In [32]:
# Preview the Dataset
df.head()

,OrderDate,CustomerID,OrderID,ProductID,ProductName,Category,Gender,Age,Quantity,ExpectedPrice,FinalPrice,Discount,Rating,Region,PaymentMethod
0,2022-04-13,420c61e3-2071-45a7-b522-0e23deca877b,0d4f06bd-4685-4f51-84da-6c69085623a8,PROD-ALPHA1409,Alpha SV Jacket,Shell,NaN,35,1,768.21,766.92,0.00,2.0,Europe,PayPal
1,2022-12-15,ec3fd6f7-7e98-4328-9138-e154bb599e2d,6322f32d-21ba-4fe7-8b91-77a44b29ecec,PROD-SENTINEL2424,Sentinel LT Jacket,Insulated,NaN,45,2,520.21,532.68,-0.02,1.0,North America,Credit Card
2,2022-09-28,ed1bf901-3a3c-40ec-b621-140d242e6a6a,b7181f59-9d30-4b02-88f4-6c6af260712a,PROD-SABRE9279,Sabre AR Pants,Shell,Non-binary,19,2,495.86,489.59,0.01,5.0,Europe,Apple Pay
3,2022-04-17,68fdea1c-c926-4563-bd81-df3314da0e85,754b0df2-e0e7-4e8e-a61d-19663c62a885,PROD-SIDEWINDER5557,Sidewinder Pants,Shell,NaN,28,3,402.38,401.25,0.00,NaN,Online,Gift Card
4,2022-03-13,aecba884-3456-4450-a251-6f7094b6bf27,1077b5f4-689e-4a0a-bab9-3c8fbb67807c,PROD-CORMAC4527,Cormac Shirt,Mid Layer,NaN,24,1,408.04,385.47,0.06,1.0,Online,Credit Card


In [33]:
# Overview of the dataset’s structure and missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OrderDate      10500 non-null  object 
 1   CustomerID     10500 non-null  object 
 2   OrderID        10500 non-null  object 
 3   ProductID      10500 non-null  object 
 4   ProductName    10500 non-null  object 
 5   Category       10500 non-null  object 
 6   Gender         6574 non-null   object 
 7   Age            10500 non-null  int64  
 8   Quantity       10500 non-null  int64  
 9   ExpectedPrice  10500 non-null  float64
 10  FinalPrice     10500 non-null  float64
 11  Discount       10500 non-null  float64
 12  Rating         8727 non-null   float64
 13  Region         10500 non-null  object 
 14  PaymentMethod  10500 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 1.2+ MB


In [34]:
# Note: 'Gender' and 'Rating' columns contain missing values that require cleaning
df.isnull().sum()

OrderDate           0
CustomerID          0
OrderID             0
ProductID           0
ProductName         0
Category            0
Gender           3926
Age                 0
Quantity            0
ExpectedPrice       0
FinalPrice          0
Discount            0
Rating           1773
Region              0
PaymentMethod       0
dtype: int64

### Data Cleaning

In [35]:
# Convert ID columns to string type
df['CustomerID'] = df['CustomerID'].astype(str)
df['OrderID'] = df['OrderID'].astype(str)
df['ProductID'] = df['ProductID'].astype(str)

In [36]:
# Standardize gender column
standardGender = {'M': 'Male', 'F': 'Female', None: 'Unknown'}
df['Gender'] = df['Gender'].str.strip().replace(standardGender)

In [ ]:
# Convert 'Rating' to numeric, coercing errors to NaN, then fill missing values with 0
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)

In [39]:
# Re-check missing values and data types after cleaning
df.isnull().sum()

OrderDate        0
CustomerID       0
OrderID          0
ProductID        0
ProductName      0
Category         0
Gender           0
Age              0
Quantity         0
ExpectedPrice    0
FinalPrice       0
Discount         0
Rating           0
Region           0
PaymentMethod    0
dtype: int64

In [40]:
# Check if there are any duplicate rows
df.duplicated().sum()

np.int64(0)

### Create database and table for data to live

In [60]:
# Connect to SQLite database
conn = sqlite3.connect('arcteryx_products.db')

# Write the cleaned DataFrame to a new SQL table, replacing existing one (if already exists)
df.to_sql('cleanedTable', conn, if_exists='replace', index=False)

# Preview dataset
result = pd.read_sql('SELECT * FROM cleanedTable', conn)
print(result)

conn.close()

        OrderDate                            CustomerID  \
0      2022-04-13  420c61e3-2071-45a7-b522-0e23deca877b   
1      2022-12-15  ec3fd6f7-7e98-4328-9138-e154bb599e2d   
2      2022-09-28  ed1bf901-3a3c-40ec-b621-140d242e6a6a   
3      2022-04-17  68fdea1c-c926-4563-bd81-df3314da0e85   
4      2022-03-13  aecba884-3456-4450-a251-6f7094b6bf27   
...           ...                                   ...   
10495  2024-12-18  d8a4b1c9-1713-4d66-a634-7bf20fb8e5c3   
10496  2024-04-10  ecdb4deb-d93b-45ff-bee9-cc10b8fafd13   
10497  2024-01-01  86d646ac-1f1f-46d8-8dd7-43e4af33865b   
10498  2024-12-16  34e8a3ec-67b7-465b-9a6a-c23e784dced0   
10499  2024-01-07  3e47d06e-e910-416d-8049-3677187a5a91   

                                    OrderID            ProductID  \
0      0d4f06bd-4685-4f51-84da-6c69085623a8       PROD-ALPHA1409   
1      6322f32d-21ba-4fe7-8b91-77a44b29ecec    PROD-SENTINEL2424   
2      b7181f59-9d30-4b02-88f4-6c6af260712a       PROD-SABRE9279   
3      754b0df2-e0e

#### Note: Due to limitations with the free version of Tableau (which does not support direct connections to SQL databases), the required data was exported into an Excel file ('arcteryxDataCleaned.xlsx').